In [1]:
from keras.applications import VGG16
#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(50, 50, 3))

Using TensorFlow backend.


58892288/58889256 [==============================] - 218s 4us/step


In [2]:
# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x000002EB53C51978> False
<keras.layers.convolutional.Conv2D object at 0x000002EB54C47BA8> False
<keras.layers.convolutional.Conv2D object at 0x000002EB600A61D0> False
<keras.layers.pooling.MaxPooling2D object at 0x000002EB600A67F0> False
<keras.layers.convolutional.Conv2D object at 0x000002EB600D0C18> False
<keras.layers.convolutional.Conv2D object at 0x000002EB600F7630> False
<keras.layers.pooling.MaxPooling2D object at 0x000002EB6010C1D0> False
<keras.layers.convolutional.Conv2D object at 0x000002EB6011B550> False
<keras.layers.convolutional.Conv2D object at 0x000002EB60130D68> False
<keras.layers.convolutional.Conv2D object at 0x000002EB60141860> False
<keras.layers.pooling.MaxPooling2D object at 0x000002EB60162278> False
<keras.layers.convolutional.Conv2D object at 0x000002EB601735F8> False
<keras.layers.convolutional.Conv2D object at 0x000002EB60187E10> False
<keras.layers.convolutional.Conv2D object at 0x000002EB60199908> False
<ker

In [7]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 2050      
Total params: 15,242,050
Trainable params: 7,606,786
Non-trainable params: 7,635,264
_________________________________________________________________


In [8]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img
image_size = 50
train_dir = './datasets/training_data'
validation_dir = './datasets/testing_data'
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 100
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 1000 images belonging to 2 classes.
Found 400 images belonging to 2 classes.


In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)
 
# Save the model
model.save('small_last4.h5')

Epoch 1/30
10/10 [==============================] - 156s 16s/step - loss: 0.6992 - acc: 0.5490 - val_loss: 0.6220 - val_acc: 0.6425
Epoch 2/30
10/10 [==============================] - 151s 15s/step - loss: 0.6526 - acc: 0.6480 - val_loss: 0.5572 - val_acc: 0.7550
Epoch 3/30
10/10 [==============================] - 151s 15s/step - loss: 0.5726 - acc: 0.7060 - val_loss: 0.5777 - val_acc: 0.7000
Epoch 4/30
10/10 [==============================] - 150s 15s/step - loss: 0.5774 - acc: 0.6970 - val_loss: 0.5232 - val_acc: 0.7800
Epoch 5/30
10/10 [==============================] - 143s 14s/step - loss: 0.5430 - acc: 0.7390 - val_loss: 0.5100 - val_acc: 0.7625
Epoch 6/30
10/10 [==============================] - 144s 14s/step - loss: 0.5132 - acc: 0.7450 - val_loss: 0.4618 - val_acc: 0.7775
Epoch 7/30
10/10 [==============================] - 145s 14s/step - loss: 0.5208 - acc: 0.7640 - val_loss: 0.5073 - val_acc: 0.7425
Epoch 8/30
 4/10 [===========>..................] - ETA: 1:08 - loss: 0.4897

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

In [ ]:
# Create a generator for prediction
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)
 
# Get the filenames from the generator
fnames = validation_generator.filenames
 
# Get the ground truth from generator
ground_truth = validation_generator.classes
 
# Get the label to class mapping from the generator
label2index = validation_generator.class_indices
 
# Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.items())
 
# Get the predictions from the model using the generator
predictions = model.predict_generator(validation_generator, steps=validation_generator.samples/validation_generator.batch_size,verbose=1)
predicted_classes = np.argmax(predictions,axis=1)
 
errors = np.where(predicted_classes != ground_truth)[0]
print("No of errors = {}/{}".format(len(errors),validation_generator.samples))
 
# Show the errors
for i in range(len(errors)):
    pred_class = np.argmax(predictions[errors[i]])
    pred_label = idx2label[pred_class]
     
    title = 'Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
        fnames[errors[i]].split('/')[0],
        pred_label,
        predictions[errors[i]][pred_class])
     
    original = load_img('{}/{}'.format(validation_dir,fnames[errors[i]]))
    plt.figure(figsize=[7,7])
    plt.axis('off')
    plt.title(title)
    plt.imshow(original)
    plt.show()

In [ ]:
from keras.models import load_model
from keras.preprocessing.image import img_to_array, load_img
severity_model = load_model('small_last4.h5')
def prepare_img_256(img_path):
    #urllib.request.urlretrieve(img_path, 'save.jpg') # or other way to upload image
    img = load_img(img_path, target_size=(50, 50)) # this is a PIL image 
    x = img_to_array(img) # this is a Numpy array with shape (3, 256, 256)
     
    x = x.reshape((1,) + x.shape)/255
    
    return x


img_2561 = prepare_img_256('../datasets/Test Image/Test Folder/Red Swift with damage.jpg')
#img_2562 = prepare_img_256('../datasets/Test Image/Test Folder/SwiftF2.jpg')
#images = np.vstack([img_2561, img_2562])
images = np.vstack([img_2561])

plt.imshow(img_2561[0])
plt.show()  
#plt.imshow(img_2562[0])
#plt.show()

pred = severity_model.predict(images)[0][0]
print(pred)